In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import random
import logging

# Configure logging
logging.basicConfig(filename='scraper.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Path to the ChromeDriver executable
driver_path = 'D:\\hp5cd\\New folder\\Web_Scraping\\chromedriver-win64\\chromedriver.exe'

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

# Optionally, disable headless mode if used earlier
# chrome_options.add_argument("--headless")

# Create a Service object with the path to the ChromeDriver
service = Service(driver_path)

# Create a new instance of the Chrome driver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Make Selenium look like a real user
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Function to simulate human-like behavior
def human_like_behavior(driver):
    # Random mouse movements
    for _ in range(random.randint(3, 7)):
        x_offset = random.randint(0, 500)
        y_offset = random.randint(0, 500)
        webdriver.ActionChains(driver).move_by_offset(x_offset, y_offset).perform()
        time.sleep(random.uniform(0.1, 0.5))

    # Scroll the page randomly
    scroll_length = random.randint(0, 500)
    driver.execute_script(f"window.scrollTo(0, {scroll_length});")
    time.sleep(random.uniform(0.5, 1.5))

# Initialize an empty list to store the data
data = []

# Loop over the pages
for page_num in range(1, 10):
    # The URL of the website, with the page number included
    url = f'https://www.pricesmart.com/en-jm/category/Groceries/G10D03?page={page_num}'

    try:
        # Navigate to the page
        driver.get(url)

        # Perform human-like behavior
        human_like_behavior(driver)

        # Wait until the `result-card` elements are present
        WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'result-card'))
        )

        # Get the page source
        html = driver.page_source

        # Parse the HTML content of the page with BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Find all divs with class 'result-card'
        divs = soup.find_all('div', class_='result-card')

        if divs:
            logging.info(f"Page {page_num}: Found {len(divs)} result cards")
            for div in divs:
                logging.info(f"Result Card: {div}")
        else:
            logging.info(f"Page {page_num}: No result cards found")

    except Exception as e:
        logging.error(f"Error loading page {page_num}: {e}")

    # Introduce a random delay between requests to avoid detection
    time.sleep(random.uniform(5, 15))

# Close the driver
driver.quit()